In [265]:
import urllib, string, time
import urllib.parse
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from astropy.time import Time
import numpy as np
from tabulate import tabulate
from astroquery.simbad import Simbad

from bokeh.plotting import output_notebook, figure, show
from bokeh.models import HoverTool, tools,ColumnDataSource
from bokeh.palettes import viridis, inferno
from bokeh.models import HoverTool, tools,ColumnDataSource, Whisker, ColorBar, LinearColorMapper
output_notebook()

Loading BokehJS ...

In [266]:
namecluster = 'NGC 6341'
result_table = Simbad.query_object(namecluster)
print(result_table)
t = result_table[0]


MAIN_ID      RA         DEC     ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"     "d:m:s"   ...                                            
------- ----------- ----------- ... -------- -------------- -------------------
  M  92 17 17 07.39 +43 08 09.4 ...        C              O 2010AJ....140.1830G


In [267]:
racluster = t['RA']
deccluster = t['DEC']
c = SkyCoord(f'{racluster} {deccluster}', unit=(u.hourangle, u.deg))

ra = c.ra.deg
dec= c.dec.deg
radius = 3.0

# create URL with desired search parameters

url = "https://archive.stsci.edu/hst/search.php?"
url = url + f"&RA={ra}"
url = url + f"&DEC={dec}"
url = url + f"&radius={radius}"
url = url + "&outputformat=CSV"
url = url + "&action=Search"


data = pd.read_csv(url,low_memory=False)

data = data.drop(data.index[0])
#m4data
#data['YearDay'] = data['Start Time'].map(lambda times: '.'.join(Time(times).yday.split(':',2)[:2]))

#data['Week'] = data['Start Time'].map(lambda times: int(np.floor(float(Time(times).yday.split(':')[1])/7.)))
#data2 = data[data['High-Level Science Products'] == '1']
t =data.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','Proposal ID']).size().reset_index().rename(columns={0:'ocurrence'})
#t.sort_values(by=['ocurrence'],ascending=False)
print(tabulate(t.sort_values(by=['ocurrence'],ascending=False), tablefmt="pipe", headers="keys"))

|     | Target Name     | Filters/Gratings            | Instrument   |   Exp Time |   Proposal ID |   ocurrence |
|----:|:----------------|:----------------------------|:-------------|-----------:|--------------:|------------:|
|  60 | NGC6341         | F555W                       | WFPC2        |     10     |         11077 |          76 |
|  61 | NGC6341         | F555W                       | WFPC2        |    100     |         11077 |          36 |
|  65 | NGC6341         | F555W                       | WFPC2        |    400     |         11077 |          26 |
|  10 | DARK            | MIRVIS                      | STIS         |     60     |         11844 |          12 |
|   4 | BIAS            | MIRVIS                      | STIS         |      0     |         11846 |          10 |
|  74 | NGC6341         | F555W;CLEAR2S               | ACS          |     20     |         10443 |           9 |
|  77 | NGC6341         | F555W;CLEAR2S               | ACS          |     40     |     

In [295]:
data['MJD'] = data['Start Time'].map(lambda times: Time(times).datetime64)
data['year'] = data['Start Time'].map(lambda times: Time(times).yday.split(':')[0])
#data['YearDay'] = data['Start Time'].map(lambda times: '.'.join(Time(times).yday.split(':',2)[:2]))


datacleaner =data[(data['Target Name'] != 'DARK') & (data['Target Name'] != 'BIAS') & (data['Target Name'] != 'EARTH-CALIB')]
dataf555w  = datacleaner[datacleaner['Filters/Gratings'] == 'F555W']

t2 =dataf555w.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','year']).size().reset_index().rename(columns={0:'ocurrence'})
t2.sort_values(by=['ocurrence'],ascending=False)


,Target Name,Filters/Gratings,Instrument,Exp Time,year,ocurrence
2,NGC6341,F555W,WFPC2,10.000,2008,76
3,NGC6341,F555W,WFPC2,100.000,2008,36
7,NGC6341,F555W,WFPC2,400.000,2008,26
10,NGC6341,F555W,WFPC2,70.000,1995,6
4,NGC6341,F555W,WFPC2,2.000,1995,4
6,NGC6341,F555W,WFPC2,40.000,2001,4
5,NGC6341,F555W,WFPC2,300.000,2001,3
8,NGC6341,F555W,WFPC2,500.000,1995,2
9,NGC6341,F555W,WFPC2,600.000,1995,2
0,NGC6341,F555W,WFC3,1360.480,2009,1


In [297]:
dataf555w  = datacleaner[(datacleaner['Filters/Gratings'] == 'F555W')&(datacleaner['Instrument'] == 'WFPC2 ') 
                         &(datacleaner['year'] == '2008')]

timelist = dataf555w['MJD'].tolist()
exptime = dataf555w['Exp Time'].to_numpy(dtype='float')
namesobs = dataf555w['Start Time'].tolist()

datatoplot = {'time':timelist,
       'y':exptime,
        'ones':[1.]*len(exptime),
       'names':namesobs}

source = ColumnDataSource(data=datatoplot)

p = figure(plot_width=900, plot_height=500, title='a',active_drag='pan', active_scroll='wheel_zoom',
          x_axis_type='datetime',y_axis_label='ExpTime (s)',x_axis_label='Date_Obs')


#Tool to get wavelength
hover2 = HoverTool(
        tooltips=[
            ('Date', '(@names)'),
            ("EXPTime",'(@y)')
        ]
    )

        
p.add_tools(hover2)



# add a circle renderer with a size, color, and alpha
#p.circle('time','y', source=source)

p.add_tools(hover2)


colorlist = inferno(len(exptime))

mapper = LinearColorMapper(palette=colorlist, low=exptime.min(), high=exptime.max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0.5),title='ExpTime')
p.circle('time','ones', source=source,color={'field': 'y', 'transform': mapper})
p.add_layout(color_bar, 'right')
show(p)

In [278]:
dataf555w['Instrument']

25     WFPC  
58     WFC3  
172    WFPC2 
173    WFPC2 
174    WFPC2 
        ...  
375    WFPC2 
376    WFPC2 
377    WFPC2 
378    WFPC2 
379    WFPC2 
Name: Instrument, Length: 161, dtype: object

In [259]:
dataf555w['Exp Time'].to_numpy(dtype='float').min()

2.0